# Busca em grafo - algoritmo A*


#### Referências

- https://www.redblobgames.com/pathfinding/a-star/introduction.html
- https://www.redblobgames.com/pathfinding/a-star/implementation.html


#### Código


In [1]:
import numpy as np
from itertools import product


In [12]:
class A_star:
    def __init__(self, dimension: tuple[int, int]) -> None:
        self.dimension = dimension
        self.x_values: list[int] = list(range(1, self.dimension[0] + 1))
        self.y_values: list[int] = list(range(1, self.dimension[1] + 1))
        self.xy_pairs: list[tuple[int, int]] = list(
            product(self.x_values, self.y_values)
        )
        self._iterations: list[tuple[int, int]] = [(1, 0), (-1, 0), (0, 1), (0, -1)]

    def reset_nodes(self) -> None:
        self.nodes: list[dict] = []
        for x, y in self.xy_pairs:
            self.nodes.append(self.gen_nodes(x, y))
        self.nodes[self.start_id]["percorrida"] = 0
        self.nodes[self.start_id]["geometrica"] = self.h_cost(self.start_id)

    def reset_sets(self) -> None:
        self.close: set[int] = set()
        self.open: set[int] = set()
        self.open.add(self.start_id)

    def set_problem(self, start: tuple[int, int], target: tuple[int, int]) -> None:
        self.start = start
        self.target = target
        self.start_id = self.xy_pairs.index(self.start)
        self.target_id = self.xy_pairs.index(self.target)

        self.reset_nodes()
        self.reset_sets()

    def gen_nodes(self, x: int, y: int) -> dict:
        node_id = self.xy_pairs.index((x, y))
        node = {
            "id": node_id,
            "x": x,
            "y": y,
            "neighbors": self.gen_neighbors(node_id, x, y),
            "percorrida": 0,
            "geometrica": self.h_cost(node_id),
            "predecessor": None,
        }
        return node

    def inside_grid(self, index_x: int, index_y: int) -> bool:
        return 0 <= index_x < self.dimension[0] and 0 <= index_y < self.dimension[1]

    def distance_grid(self, node_id: int, neighbor_id: int) -> float:
        x, y = self.xy_pairs[node_id]
        xn, yn = self.xy_pairs[neighbor_id]

        if (y, yn) in [(5, 6), (6, 5)] and (x, xn) in [
            (3, 3),
            (4, 4),
            (5, 5),
            (6, 6),
            (7, 7),
        ]:
            return np.inf

        if (x, xn) in [(5, 6), (6, 5), (7, 8), (8, 7)]:
            return 5

        if (y, yn) in [(5, 6), (6, 5)]:
            return 15

        return 10

    def gen_neighbors(self, node_id: int, x: int, y: int) -> dict:
        index_x = self.x_values.index(x)
        index_y = self.y_values.index(y)

        neighbors = dict()
        for xi, yi in self._iterations:
            new_index_x = index_x + xi
            new_index_y = index_y + yi

            if not self.inside_grid(new_index_x, new_index_y):
                continue

            neighbor = (
                self.x_values[new_index_x],
                self.y_values[new_index_y],
            )
            neighbor_id = self.xy_pairs.index(neighbor)
            if (distancia := self.distance_grid(node_id, neighbor_id)) == np.inf:
                continue
            neighbors[neighbor_id] = distancia
        return neighbors

    def get_neighbors(self, id: int) -> list[int]:
        yield from [n for n in self.nodes[id]["neighbors"]]

    def get_route(self) -> list[tuple[int, int]]:
        route = []
        current = self.target_id
        percorrida = self.g_cost(current)
        geometrica = self.get_geometrica(current)
        f_eval = percorrida + geometrica
        route.append(
            (
                self.coord_conversion(self.xy_pairs[current]),
                percorrida,
                geometrica,
                f_eval,
            )
        )

        while current != self.start_id:
            current = self.nodes[current]["predecessor"]
            percorrida = self.g_cost(current)
            geometrica = self.get_geometrica(current)
            f_eval = percorrida + geometrica
            route.append(
                (
                    self.coord_conversion(self.xy_pairs[current]),
                    percorrida,
                    geometrica,
                    f_eval,
                )
            )

        route.reverse()

        return route

    def h_cost(self, id: int) -> float:
        """Distância geometrica entre o nó atual e o target"""
        x, y = self.xy_pairs[id]
        x = x * 10
        y = y * 10
        
        if x >= 60:
            x -= 5
        if x >= 75:
            x -= 5
        if y <= 50:
            y += 5

        return ((x - 10 * self.target[0]) ** 2 + (y - 10 * self.target[1]) ** 2) ** 0.5

    def g_cost(self, id: int) -> float:
        """Distância do percurso até esse ponto"""
        return self.nodes[id]["percorrida"]

    def movement_cost(self, id: int, n_id: int) -> float:
        """Distância que o movimento para o próximo ponto vai adicionar"""
        return self.nodes[id]["neighbors"][n_id]

    def best_n_from_open(self) -> int:
        return min(
            self.open,
            key=lambda id: self.g_cost(id) + self.get_geometrica(id),
        )

    def coord_conversion(self, coord: tuple[int, int]) -> tuple[chr, int]:
        return chr(coord[0] + 64) + str(coord[1])

    def show_iteration(self, n_id):
        n = self.coord_conversion(self.xy_pairs[n_id])
        f_val = f'{self.g_cost(n_id) + self.get_geometrica(n_id):0.2f}'
        print(
            n,
            # self.g_cost(n_id),
            # self.get_geometrica(n_id),
            f_val,
            end=" ",
        )

    def get_geometrica(self, id):
        return self.nodes[id]["geometrica"]

    def execute(self):
        current = self.start_id
        while current != self.target_id:
            current = self.best_n_from_open()
            self.open.discard(current)
            self.close.add(current)
            h_cost_current = self.get_geometrica(current)

            print(self.coord_conversion(self.xy_pairs[current]), end=" ")

            for n in self.get_neighbors(current):
                cost = self.g_cost(current) + self.movement_cost(current, n)

                # Caso ele tenha conseguido fazer um caminho mais eficiênte até esse ponto
                # ou seja a primeira vez que ele passa por esse ponto
                if self.g_cost(n) > cost or (
                    (n not in self.close) and (n not in self.open)
                ):
                    h_cost = self.get_geometrica(n)

                    # Considera só as movimentações em direção a chegada
                    if h_cost < h_cost_current:
                        self.close.discard(n)
                        self.nodes[n]["percorrida"] = cost
                        self.nodes[n]["predecessor"] = current
                        self.open.add(n)

                        self.show_iteration(n)
            print()


In [13]:
problem = A_star((10, 10))
problem.set_problem((9, 2), (1, 9))
problem.execute()

I2 H2 98.46 I3 99.02 
H2 G2 100.15 H3 101.39 
I3 I4 103.22 
G2 F2 104.06 G3 102.78 
H3 H4 105.00 
G3 F3 106.06 G4 106.06 
I4 I5 108.26 
F2 E2 106.32 
H4 H5 109.46 
F3 E3 108.01 F4 108.64 
G4 G5 110.19 
E2 D2 111.59 
E3 D3 112.65 E4 110.21 
I5 I6 121.16 
F4 F5 112.01 
H5 H6 122.08 
G5 
E4 D4 114.08 E5 113.15 
D2 C2 118.01 
F5 
D3 C3 118.52 
E5 D5 116.10 
D4 C4 119.24 
D5 C5 120.31 
C2 B2 125.76 
C3 B3 125.90 
C4 B4 126.10 
C5 B5 126.40 
I6 I7 127.80 
H6 G6 122.65 H7 128.25 
G6 F6 124.08 G7 128.52 
F6 E6 125.00 F7 129.24 
E6 D6 127.43 E7 129.72 
B2 A2 135.00 
B3 A3 135.00 
B4 A4 135.00 
B5 A5 135.00 B6 136.62 
D6 C6 131.06 D7 131.06 
I7 I8 135.71 
H7 H8 135.83 
G7 G8 135.90 
F7 F8 136.10 
E7 E8 136.23 
C6 C7 133.28 
D7 D8 136.62 
C7 B7 137.36 C8 137.36 
A2 
A3 
A4 
A5 A6 145.00 
I8 I9 145.00 
H8 H9 145.00 
G8 G9 145.00 
F8 F9 145.00 
E8 E9 145.00 
B6 
D8 D9 145.00 
B7 A7 145.00 B8 139.14 
C8 C9 145.00 
B8 A8 145.00 B9 145.00 
G9 
A6 
A7 
A8 A9 145.00 
A9 


In [4]:
problem.get_route()

[('I2', 0, 95.524865872714, 95.524865872714),
 ('H2', 10, 88.45903006477066, 98.45903006477066),
 ('H3', 20, 81.39410298049853, 101.39410298049853),
 ('H4', 30, 75.0, 105.0),
 ('H5', 40, 69.46221994724903, 109.46221994724903),
 ('H6', 55, 67.08203932499369, 122.08203932499369),
 ('G6', 60, 62.64982043070834, 122.64982043070833),
 ('F6', 70, 54.08326913195984, 124.08326913195984),
 ('E6', 75, 50.0, 125.0),
 ('D6', 85, 42.42640687119285, 127.42640687119285),
 ('C6', 95, 36.05551275463989, 131.0555127546399),
 ('C7', 105, 28.284271247461902, 133.2842712474619),
 ('B7', 115, 22.360679774997898, 137.3606797749979),
 ('B8', 125, 14.142135623730951, 139.14213562373095),
 ('A8', 135, 10.0, 145.0),
 ('A9', 145, 0.0, 145.0)]

In [5]:
# Conferência manual utilizada
def trans_xy(x, y):
    x = x * 10
    y = y * 10

    if x >= 60:
        x -= 5

    if x >= 75:
        x -= 5

    if y <= 50:
        y += 5
    return x, y

x, y = trans_xy(8,4)
d_percurso = 15
d_movimento = 15

d_objetivo = ((x - 10) ** 2 + (y - 90) ** 2) ** 0.5
f_aval = d_objetivo + d_movimento + d_percurso
print(d_movimento+d_percurso) 
print(d_objetivo)
print(f_aval)

30
75.0
105.0
